In [50]:
import numpy as np
import pandas as pd

In [114]:
data = pd.read_csv('input.csv',sep='\t',header=None)
data = data.rename(columns = {0:'input',1:'classes'})
data.dropna(inplace = True)
# data

In [115]:
from sklearn.model_selection import train_test_split

X = data['input']
y = data['classes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [117]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler


# vectorizer = TfidfVectorizer()
# X_train = vectorizer.fit_transform(X_train)
# y_train = vectorizer.fit_transform(y_train)

def create_model():
    # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
    # equal to number of intents to predict output intent with softmax
    model = Sequential()
    model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(y_train[0]), activation='softmax'))

    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model
    
    
# wrap the model using the function you created
clf = KerasClassifier(build_fn=create_model,verbose=0)

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('ss', StandardScaler(with_mean=False,)), 
                     ('clf', clf)
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 
# print(clf)

InvalidArgumentError: indices[3] = [1,16] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

In [94]:
type(y_train)

scipy.sparse.csr.csr_matrix

In [60]:
# Form a prediction set
predictions = text_clf.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

[[0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 2 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 2 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 2 0 0 0 0]
 [0 0 3 0 0 0 0 0]]
              precision    recall  f1-score   support

       Books       0.00      0.00      0.00         1
     Goodbye       0.00      0.00      0.00         1
    Greeting       0.25      1.00      0.40         2
       Learn       0.25      1.00      0.40         1
       Music       0.00      0.00      0.00         2
     Options       1.00      1.00      1.00         1
       Shows       0.00      0.00      0.00         2
      Thanks       0.00      0.00      0.00         3

    accuracy                           0.31        13
   macro avg       0.19      0.38      0.23        13
weighted avg       0.13      0.31      0.17        13

0.3076923076923077


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
